In [1]:
#import dependencies

import pandas as pd
from sqlalchemy import create_engine
from config import username, password

In [2]:
#Extract vaccineData csv and store to a dataframe
pd.set_option('display.max_columns', None)
vaccineData_file = "static/data/vaccineData.csv"
vaccine_df = pd.read_csv(vaccineData_file)
vaccine_df.head()

,date,country,state,county,fips,lat,long,locationId,actuals.cases,actuals.deaths,actuals.positiveTests,actuals.negativeTests,actuals.contactTracers,actuals.hospitalBeds.capacity,actuals.hospitalBeds.currentUsageTotal,actuals.hospitalBeds.currentUsageCovid,actuals.hospitalBeds.typicalUsageRate,actuals.icuBeds.capacity,actuals.icuBeds.currentUsageTotal,actuals.icuBeds.currentUsageCovid,actuals.icuBeds.typicalUsageRate,actuals.newCases,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,metrics.testPositivityRatio,metrics.testPositivityRatioDetails,metrics.caseDensity,metrics.contactTracerCapacityRatio,metrics.infectionRate,metrics.infectionRateCI90,metrics.icuHeadroomRatio,metrics.icuHeadroomDetails,metrics.icuCapacityRatio,riskLevels.overall,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered
0,3/1/2020,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,3/2/2020,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2,3/3/2020,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3,3/4/2020,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,3/5/2020,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


In [3]:
#drop unwanted and unneccessary columns

vaccine_df = vaccine_df.loc[:, ['date', 'country', 'state', 'actuals.cases', 'actuals.newCases', 'actuals.vaccinesDistributed', 'actuals.vaccinationsInitiated', 
                               'actuals.vaccinationsCompleted', 'metrics.infectionRate']]

vaccine_df.head(400)

,date,country,state,actuals.cases,actuals.newCases,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,metrics.infectionRate
0,3/1/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
1,3/2/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
2,3/3/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
3,3/4/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
4,3/5/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
395,3/31/2021,US,AK,62559.0,164.0,601225.0,248711.0,163100.0,1.075456
396,4/1/2021,US,AK,62785.0,226.0,615825.0,250378.0,165325.0,1.069876
397,4/2/2021,US,AK,62987.0,202.0,629315.0,252586.0,167561.0,1.063177
398,4/3/2021,US,AK,62987.0,0.0,636665.0,254263.0,170747.0,1.054094


In [4]:
#Rename columns for sql compli

vaccine_df.rename(columns={"date":"date","country": "country", "state": "state", 
                           "actuals.cases": "cases", "actuals.newCases": "new_cases", 
                           "actuals.vaccinesDistributed":"vaccines_distributed", 
                           "actuals.vaccinationsInitiated": "vaccines_initiated", 
                           "actuals.vaccinationsCompleted":"vaccines_completed",
                           "metrics.infectionRate": "infection_rate"}, inplace=True)

vaccine_df.head(100)

,date,country,state,cases,new_cases,vaccines_distributed,vaccines_initiated,vaccines_completed,infection_rate
0,3/1/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
1,3/2/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
2,3/3/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
3,3/4/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
4,3/5/2020,US,AK,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,6/4/2020,US,AK,542.0,26.0,NaN,NaN,NaN,1.281076
96,6/5/2020,US,AK,565.0,23.0,NaN,NaN,NaN,1.250160
97,6/6/2020,US,AK,579.0,14.0,NaN,NaN,NaN,1.218703
98,6/7/2020,US,AK,588.0,9.0,NaN,NaN,NaN,1.187972


In [5]:
vaccine_df.dtypes

date                     object
country                  object
state                    object
cases                   float64
new_cases               float64
vaccines_distributed    float64
vaccines_initiated      float64
vaccines_completed      float64
infection_rate          float64
dtype: object

In [6]:
#create connection to sql and engine to run

rds_connection_string = f"{username}:{password}@localhost:5432/vaccineData_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
engine.table_names()

['vaccinations']

In [8]:
vaccine_df.to_sql(name='vaccinations', con=engine, if_exists='append', index=False)